## Thermal block problem
**_Keywords: certified reduced basis method, scalar problem_**

### 1. Introduction
In this Tutorial, we consider steady heat conduction in a two-dimensional domain $\Omega$.

We define two subdomains $\Omega_1$ and $\Omega_2$, such that
1. $\Omega_1$ is a disk centered at the origin of radius $r_0=0.5$, and
2. $\Omega_2=\Omega/\ \overline{\Omega_1}$. 

The conductivity $\kappa$ is assumed to be constant on $\Omega_1$ and $\Omega_2$, i.e.
$$
\kappa|_{\Omega_1}=\kappa_0 \quad \textrm{and} \quad \kappa|_{\Omega_2}=1.
$$

For this problem, we consider $P=2$ parameters:
1. the first one is related to the conductivity in $\Omega_1$, i.e. $\mu_0\equiv k_0$ (_note that parameters numbering is zero-based_);
2. the second parameter $\mu_1$ takes into account the constant heat flux over $\Gamma_{base}$.

The parameter vector $\boldsymbol{\mu}$ is thus given by 
$$
\boldsymbol{\mu} = (\mu_0,\mu_1)
$$
on the parameter domain
$$
\mathbb{P}=[0.1,10]\times[-1,1].
$$

In this problem we model the heat transfer process due to the heat flux over the bottom boundary $\Gamma_{base}$ and the following conditions on the remaining boundaries:
* the left and right boundaries $\Gamma_{side}$ are insulated,
* the top boundary $\Gamma_{top}$ is kept at a reference temperature (say, zero),

with the aim of measuring the average temperature on $\Gamma_{base}$.

In order to obtain a faster evaluation (yet, provably accurate) of the output of interest we propose to use a certified reduced basis approximation for the problem.

### 2. Parametrized formulation

Let $u(\boldsymbol{\mu})$ be the temperature in the domain $\Omega$.

The strong formulation of the parametrized problem is given by: for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})$ such that

$$
\begin{cases}
	- \text{div} (\kappa(\mu_0)\nabla u(\boldsymbol{\mu})) = 0 & \text{in } \Omega,\\
	u(\boldsymbol{\mu}) = 0 & \text{on } \Gamma_{top},\\
	\kappa(\mu_0)\nabla u(\boldsymbol{\mu})\cdot \mathbf{n} = 0 & \text{on } \Gamma_{side},\\
	\kappa(\mu_0)\nabla u(\boldsymbol{\mu})\cdot \mathbf{n} = \mu_1 & \text{on } \Gamma_{base}.
\end{cases}
$$
<br>

where 
* $\mathbf{n}$ denotes the outer normal to the boundaries $\Gamma_{side}$ and $\Gamma_{base}$,
* the conductivity $\kappa(\mu_0)$ is defined as follows:
$$
\kappa(\mu_0) =
\begin{cases}
	\mu_0 & \text{in } \Omega_1,\\
	1 & \text{in } \Omega_2,\\
\end{cases}
$$

The corresponding weak formulation reads: for a given parameter $\boldsymbol{\mu}\in\mathbb{P}$, find $u(\boldsymbol{\mu})\in\mathbb{V}$ such that

$$a\left(u(\boldsymbol{\mu}),v;\boldsymbol{\mu}\right)=f(v;\boldsymbol{\mu})\quad \forall v\in\mathbb{V}$$

where

* the function space $\mathbb{V}$ is defined as
$$
\mathbb{V} = \{v\in H^1(\Omega) : v|_{\Gamma_{top}}=0\}
$$
* the parametrized bilinear form $a(\cdot, \cdot; \boldsymbol{\mu}): \mathbb{V} \times \mathbb{V} \to \mathbb{R}$ is defined by
$$a(u, v;\boldsymbol{\mu})=\int_{\Omega} \kappa(\mu_0)\nabla u\cdot \nabla v \ d\boldsymbol{x},$$
* the parametrized linear form $f(\cdot; \boldsymbol{\mu}): \mathbb{V} \to \mathbb{R}$ is defined by
$$f(v; \boldsymbol{\mu})= \mu_1\int_{\Gamma_{base}}v \ ds.$$

The (compliant) output of interest $s(\boldsymbol{\mu})$ given by
$$s(\boldsymbol{\mu}) = \mu_1\int_{\Gamma_{base}} u(\boldsymbol{\mu})$$
is computed for each $\boldsymbol{\mu}$.

In [1]:
import os
import sys
sys.path.append('../../')

from mlnics import NN, Losses, Normalization, RONNData, IO, Training, ErrorAnalysis
from dolfin import *
from rbnics import *
import torch
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(0)
np.random.seed(0)
%matplotlib tk

## 3. Affine decomposition

For this problem the affine decomposition is straightforward:
$$a(u,v;\boldsymbol{\mu})=\underbrace{\mu_0}_{\Theta^{a}_0(\boldsymbol{\mu})}\underbrace{\int_{\Omega_1}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_0(u,v)} \ + \  \underbrace{1}_{\Theta^{a}_1(\boldsymbol{\mu})}\underbrace{\int_{\Omega_2}\nabla u \cdot \nabla v \ d\boldsymbol{x}}_{a_1(u,v)},$$
$$f(v; \boldsymbol{\mu}) = \underbrace{\mu_1}_{\Theta^{f}_0(\boldsymbol{\mu})} \underbrace{\int_{\Gamma_{base}}v \ ds}_{f_0(v)}.$$
We will implement the numerical discretization of the problem in the class
```
class ThermalBlock(EllipticCoerciveCompliantProblem):
```
by specifying the coefficients $\Theta^{a}_*(\boldsymbol{\mu})$ and $\Theta^{f}_*(\boldsymbol{\mu})$ in the method
```
    def compute_theta(self, term):     
```
and the bilinear forms $a_*(u, v)$ and linear forms $f_*(v)$ in
```
    def assemble_operator(self, term):
```

In [2]:
class ThermalBlock(EllipticCoerciveCompliantProblem):

    # Default initialization of members
    def __init__(self, V, **kwargs):
        # Call the standard initialization
        EllipticCoerciveCompliantProblem.__init__(self, V, **kwargs)
        # ... and also store FEniCS data structures for assembly
        assert "subdomains" in kwargs
        assert "boundaries" in kwargs
        self.subdomains, self.boundaries = kwargs["subdomains"], kwargs["boundaries"]
        self.u = TrialFunction(V)
        self.v = TestFunction(V)
        self.dx = Measure("dx")(subdomain_data=self.subdomains)
        self.ds = Measure("ds")(subdomain_data=self.boundaries)

    # Return custom problem name
    def name(self):
        return "ThermalBlock"

    # Return the alpha_lower bound.
    def get_stability_factor_lower_bound(self):
        return min(self.compute_theta("a"))

    # Return theta multiplicative terms of the affine expansion of the problem.
    def compute_theta(self, term):
        mu = self.mu
        if term == "a":
            theta_a0 = mu[0]
            theta_a1 = 1.
            return (theta_a0, theta_a1)
        elif term == "f":
            theta_f0 = mu[1]
            return (theta_f0,)
        else:
            raise ValueError("Invalid term for compute_theta().")

    # Return forms resulting from the discretization of the affine expansion of the problem operators.
    def assemble_operator(self, term):
        v = self.v
        dx = self.dx
        if term == "a":
            u = self.u
            a0 = inner(grad(u), grad(v)) * dx(1)
            a1 = inner(grad(u), grad(v)) * dx(2)
            return (a0, a1)
        elif term == "f":
            ds = self.ds
            f0 = v * ds(1)
            return (f0,)
        elif term == "dirichlet_bc":
            bc0 = [DirichletBC(self.V, Constant(0.0), self.boundaries, 3)]
            return (bc0,)
        elif term == "inner_product":
            u = self.u
            x0 = inner(grad(u), grad(v)) * dx
            return (x0,)
        else:
            raise ValueError("Invalid term for assemble_operator().")

## 4. Main program
### 4.1. Read the mesh for this problem
The mesh was generated by the [data/generate_mesh.ipynb](data/generate_mesh.ipynb) notebook.

In [3]:
mesh = Mesh("data/thermal_block.xml")
subdomains = MeshFunction("size_t", mesh, "data/thermal_block_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "data/thermal_block_facet_region.xml")

### 4.2. Create Finite Element space (Lagrange P1)

In [4]:
V = FunctionSpace(mesh, "Lagrange", 1)

### 4.3. Allocate an object of the ThermalBlock class

In [5]:
problem = ThermalBlock(V, subdomains=subdomains, boundaries=boundaries)
mu_range = [(0.1, 10.0), (-1.0, 1.0)]
problem.set_mu_range(mu_range)

### 4.4. Prepare reduction with a reduced basis method

In [6]:
reduction_method = PODGalerkin(problem)
reduction_method.set_Nmax(10)
reduction_method.set_tolerance(1e-10)

### 4.5. Perform the offline phase

#### 4.5.1 Fit Reduction Method

In [7]:
reduction_method.initialize_training_set(100)
reduced_problem = reduction_method.offline()

=                ThermalBlock POD-Galerkin offline phase begins                =

###################################### 0 #######################################
truth solve for mu = (5.533253688880515, 0.43037873274483895)
update snapshots matrix

###################################### 1 #######################################
truth solve for mu = (6.067357423109274, 0.08976636599379373)
update snapshots matrix

###################################### 2 #######################################
truth solve for mu = (4.294182513455157, 0.29178822613331223)
update snapshots matrix

###################################### 3 #######################################
truth solve for mu = (4.432113391500656, 0.7835460015641595)
update snapshots matrix

###################################### 4 #######################################
truth solve for mu = (9.64026132896019, -0.2331169623484446)
update snapshots matrix

###################################### 5 #######################################

truth solve for mu = (2.965120320177291, -0.6336172759857663)
update snapshots matrix

###################################### 48 ######################################
truth solve for mu = (5.906478054619823, -0.9597849076250129)
update snapshots matrix

###################################### 49 ######################################
truth solve for mu = (8.306506289251894, -0.9906090476149059)
update snapshots matrix

###################################### 50 ######################################
truth solve for mu = (6.810383714282678, -0.4599840536156703)
update snapshots matrix

###################################### 51 ######################################
truth solve for mu = (7.378420819013689, 0.9243770902348765)
update snapshots matrix

###################################### 52 ######################################
truth solve for mu = (2.562656120847585, 0.1523146688356738)
update snapshots matrix

###################################### 53 #################################

truth solve for mu = (1.9433107582153282, 0.8887447799678672)
update snapshots matrix

###################################### 97 ######################################
truth solve for mu = (7.421552870987947, -0.019082382764865846)
update snapshots matrix

###################################### 98 ######################################
truth solve for mu = (2.3514048169359003, -0.4912870364592141)
update snapshots matrix

###################################### 99 ######################################
truth solve for mu = (0.6744886872063686, -0.13116674888375845)
update snapshots matrix

################################# perform POD ##################################
lambda_0 = 95.7978523817353
lambda_1 = 3.7984245901485156
lambda_2 = 0.00030059849588982075
lambda_3 = 2.6799038385762646e-08

build reduced operators

=                 ThermalBlock POD-Galerkin offline phase ends                 =



#### 4.5.2 Train Greedy PDNN-PINN Method

In [8]:
input_normalization_pdnn = Normalization.MinMaxNormalization(input_normalization=True)
output_normalization_pdnn = Normalization.MinMaxNormalization()

pdnn_net  = NN.RONN("PDNN", problem, reduction_method, n_hidden=2, n_neurons=40)
pdnn_loss = Losses.PDNN_Loss(pdnn_net, output_normalization_pdnn)
data      = RONNData.RONNDataLoader(pdnn_net, validation_proportion=0.2)
optimizer = torch.optim.Adam(pdnn_net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.9999)

pdnn_trainer = Training.PDNNTrainer(
    pdnn_net, data, pdnn_loss, optimizer, scheduler,
    input_normalization_pdnn, num_epochs=15000
)

loaded, starting_epoch = IO.initialize_parameters(
    pdnn_net, data, pdnn_trainer, optimizer
)

In [10]:
n = 1000 # number of parameters for which we compute residuals each iteration
num_new_params = 10 # number of parameters with which we augment data each iteration

# train PDNN
print("Initial training...")
pdnn_trainer.train()

new_parameters = pdnn_net.augment_parameters_with_time(
        RONNData.get_new_parameters(pdnn_net, n)
)

print("Assembling residuals...")
residuals, _, loss_functions = ErrorAnalysis.get_residuals(
    pdnn_net, data, new_parameters, 
    input_normalization_pdnn, output_normalization_pdnn,
    plot_residuals=True
)


print("Beginning data augmentation cycles...")
for i in range(9):
    probabilities = residuals
    probabilities /= np.sum(probabilities)
    sample_indices = np.random.choice(residuals.shape[0], size=residuals.shape[0],
                                      replace=False, p=probabilities)
    cumulative_probs = np.cumsum(probabilities[sample_indices])
    #last_index = np.searchsorted(cumulative_probs, 0.25) + 1
    #sample_indices = sample_indices[:last_index]
    sample_indices = sample_indices[:num_new_params]
    high_residual_params = new_parameters[sample_indices]
    plt.scatter(high_residual_params[:, 0].numpy().reshape(-1), 
                high_residual_params[:, 1].numpy().reshape(-1), marker='x', color='black')
    
    plt.show()
    
    # solve reduced PDE system for parameters found above and augment training data
    for j, mu in enumerate(high_residual_params):
        reduced_problem.set_mu(tuple(np.array(mu)))
        reduced_problem.solve()
        solution = torch.tensor(np.array(reduced_problem._solution.vector()[:])).view(-1, 1)
        pdnn_loss.concatenate_snapshots(solution)
        
    data.train_data = torch.cat([data.train_data, high_residual_params])
    
    # train PDNN
    pdnn_trainer.train()
    
    residuals, _, _ = ErrorAnalysis.get_residuals(
        pdnn_net, data, new_parameters, 
        input_normalization_pdnn, output_normalization_pdnn,
        loss_functions, plot_residuals=True
    )

Initial training...
150000 4.303940513578767e-05 	Loss(validation) = 7.417905205718844e-05
150100 4.303938796124883e-05 	Loss(validation) = 7.417872586531438e-05
150200 4.303936025042183e-05 	Loss(validation) = 7.417917854165849e-05
150300 4.303942093347768e-05 	Loss(validation) = 7.417904366969794e-05
150400 4.303935935121628e-05 	Loss(validation) = 7.417934043231714e-05
150500 4.303932670741411e-05 	Loss(validation) = 7.417937108710378e-05
150600 4.30392549224644e-05 	Loss(validation) = 7.417973074632975e-05
150700 4.3039216980573577e-05 	Loss(validation) = 7.417990247482158e-05
150800 4.303912576883957e-05 	Loss(validation) = 7.418031943850901e-05
150900 4.303919682489439e-05 	Loss(validation) = 7.418048015581523e-05
151000 4.3039243775324124e-05 	Loss(validation) = 7.418035336665104e-05
151100 4.303919293807607e-05 	Loss(validation) = 7.418058163095393e-05
151200 4.303915696866476e-05 	Loss(validation) = 7.418078146852509e-05
151300 4.303906121433074e-05 	Loss(validation) = 7.41808

161500 4.3038401246950274e-05 	Loss(validation) = 7.418488020140543e-05
161600 4.3038441596761305e-05 	Loss(validation) = 7.41848730317295e-05
161700 4.303844128537963e-05 	Loss(validation) = 7.418462658704937e-05
161800 4.303848562675242e-05 	Loss(validation) = 7.418455186077697e-05
161900 4.3038457304262895e-05 	Loss(validation) = 7.418438365760263e-05
162000 4.303847263569805e-05 	Loss(validation) = 7.418430441530451e-05
162100 4.303847719072987e-05 	Loss(validation) = 7.418430752407425e-05
162200 4.3038493138656175e-05 	Loss(validation) = 7.418437253438568e-05
162300 4.303853815088822e-05 	Loss(validation) = 7.418440503954123e-05
162400 4.3038526023485096e-05 	Loss(validation) = 7.418432592314449e-05
162500 4.3038471477110314e-05 	Loss(validation) = 7.418432435810649e-05
162600 4.303843169563655e-05 	Loss(validation) = 7.418437037646269e-05
162700 4.303843142479043e-05 	Loss(validation) = 7.41844402021342e-05
162800 4.3038475954620175e-05 	Loss(validation) = 7.418444597640069e-05
1

173000 4.2806666921572116e-05 	Loss(validation) = 7.418412032448968e-05
173100 4.280666622618784e-05 	Loss(validation) = 7.418412032448968e-05
173200 4.280666961546167e-05 	Loss(validation) = 7.418413449314445e-05
173300 4.2806668217210944e-05 	Loss(validation) = 7.41841344784821e-05
173400 4.280667032885865e-05 	Loss(validation) = 7.418413573006887e-05
173500 4.280666759282563e-05 	Loss(validation) = 7.418416073791127e-05
173600 4.2806667133939695e-05 	Loss(validation) = 7.418414836778557e-05
173700 4.2806671756850724e-05 	Loss(validation) = 7.418414942329009e-05
173800 4.280667557146033e-05 	Loss(validation) = 7.41841524584297e-05
173900 4.280668771807357e-05 	Loss(validation) = 7.41841524584297e-05
174000 4.280669091747456e-05 	Loss(validation) = 7.418415237433749e-05
174100 4.280669053495143e-05 	Loss(validation) = 7.418414670744823e-05
174200 4.280668888289383e-05 	Loss(validation) = 7.41841462630923e-05
174300 4.280668930006268e-05 	Loss(validation) = 7.418414873992316e-05
174400

184600 7.907434220100476e-05 	Loss(validation) = 7.418394331330558e-05
184700 7.907434220100476e-05 	Loss(validation) = 7.418394331330558e-05
184800 7.907434220100476e-05 	Loss(validation) = 7.418394331330558e-05
184900 7.907434220100476e-05 	Loss(validation) = 7.418394331330558e-05
185000 7.907434220100476e-05 	Loss(validation) = 7.418394331330558e-05
185100 7.907434220049837e-05 	Loss(validation) = 7.418394331330558e-05
185200 7.907434220049837e-05 	Loss(validation) = 7.418394331330558e-05
185300 7.907434220049837e-05 	Loss(validation) = 7.418394331330558e-05
185400 7.907434220049837e-05 	Loss(validation) = 7.418394331330558e-05
185500 7.907434220049837e-05 	Loss(validation) = 7.418394331330558e-05
185600 7.907434220049837e-05 	Loss(validation) = 7.418394331330558e-05
185700 7.907434220049837e-05 	Loss(validation) = 7.418394331330558e-05
185800 7.907434220049837e-05 	Loss(validation) = 7.418394331330558e-05
185900 7.907434220049837e-05 	Loss(validation) = 7.418394331330558e-05
186000

196200 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
196300 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
196400 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
196500 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
196600 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
196700 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
196800 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
196900 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
197000 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
197100 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
197200 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
197300 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
197400 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
197500 7.67946288462088e-05 	Loss(validation) = 7.418392621135542e-05
197600 7.67946288462

...

242600 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
242700 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
242800 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
242900 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
243000 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
243100 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
243200 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
243300 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
243400 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
243500 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
243600 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
243700 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
243800 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
243900 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
244000 0.00021405302

254400 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
254500 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
254600 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
254700 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
254800 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
254900 0.000214053029569428 	Loss(validation) = 7.418392621135542e-05
255000 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
255100 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
255200 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
255300 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
255400 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
255500 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
255600 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
255700 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
2558

265800 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
265900 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
266000 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
266100 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
266200 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
266300 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
266400 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
266500 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
266600 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
266700 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
266800 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
266900 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
267000 0.00021395241981345757 	Loss(validation) = 7.418392621135542e-05
267100 0.00021395241981345757 	Loss(validation) = 7.418392621135

277300 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
277400 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
277500 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
277600 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
277700 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
277800 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
277900 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
278000 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
278100 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
278200 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
278300 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
278400 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
278500 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
278600 0.0002685534340897094 	Loss(validation) = 7.418392621135542e-05
278700

288800 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
288900 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
289000 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
289100 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
289200 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
289300 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
289400 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
289500 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
289600 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
289700 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
289800 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
289900 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
290000 0.00025902344775863636 	Loss(validation) = 7.418392621135542e-05
290100 0.00025902344775863636 	Loss(validation) = 7.418392621135

In [11]:
fig, ax = Training.plot_loss(pdnn_trainer, pdnn_net)

#### 4.5.3 Train PRNN

In [ ]:
input_normalization_prnn = Normalization.MinMaxNormalization(input_normalization=True)
output_normalization_prnn = Normalization.MinMaxNormalization()

omega = 1.
prnn_net  = NN.RONN(f"PRNN_{omega}", problem, reduction_method, n_hidden=2, n_neurons=40)
prnn_loss = Losses.PRNN_Loss(prnn_net, output_normalization_prnn, omega=omega)
data      = RONNData.RONNDataLoader(prnn_net, validation_proportion=0.2,
                                    num_without_snapshots=213)
optimizer = torch.optim.Adam(prnn_net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.99999)

prnn_trainer = Training.PRNNTrainer(
    prnn_net, data, prnn_loss, optimizer, scheduler,
    input_normalization_prnn, num_epochs=100000
)

loaded, starting_epoch = IO.initialize_parameters(
    prnn_net, data, prnn_trainer, optimizer
)

In [ ]:
prnn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(prnn_trainer, prnn_net, separate=True)

#### 4.5.4 Train PINN

In [ ]:
input_normalization_pinn = Normalization.MinMaxNormalization(input_normalization=True)
output_normalization_pinn = Normalization.MinMaxNormalization()

pinn_net  = NN.RONN(f"PINN", problem, reduction_method, n_hidden=2, n_neurons=40)
pinn_loss = Losses.PINN_Loss(pinn_net, output_normalization_pinn)
data      = RONNData.RONNDataLoader(pinn_net, validation_proportion=0.2,
                                    num_without_snapshots=213)
optimizer = torch.optim.Adam(pinn_net.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, 0.99999)

pinn_trainer = Training.PINNTrainer(
    pinn_net, data, pinn_loss, optimizer, scheduler,
    input_normalization_pinn, num_epochs=100000
)

loaded, starting_epoch = IO.initialize_parameters(
    pinn_net, data, pinn_trainer, optimizer
)

In [ ]:
pinn_trainer.train()

In [ ]:
fig, ax = Training.plot_loss(pinn_trainer, pinn_net)

### 4.6. Perform an error analysis

In [ ]:
np.random.seed(42)
reduction_method.initialize_testing_set(100)
test_mu = torch.tensor(reduction_method.testing_set)

#### 4.6.1 Greedy PDNN-PINN Error Analysis

In [ ]:
pdnn_net.load_best_weights()
_ = ErrorAnalysis.error_analysis_fixed_net(
    pdnn_net, test_mu, input_normalization_pdnn, output_normalization_pdnn
)

#### 4.6.2 PRNN Error Analysis

In [ ]:
prnn_net.load_best_weights()
_ = ErrorAnalysis.error_analysis_fixed_net(
    prnn_net, test_mu, input_normalization_prnn, output_normalization_prnn
)

#### 4.6.3 PINN Error Analysis

In [ ]:
pinn_net.load_best_weights()
_ = ErrorAnalysis.error_analysis_fixed_net(
    pinn_net, test_mu, input_normalization_pinn, output_normalization_pinn
)